<font size="6">データ前処理について</font>

Source: https://github.com/rasbt/python-machine-learning-book


In [1]:
#グラフ描画用のおまじない
from IPython.display import Image
%matplotlib inline

In [2]:
# Added version check for recent scikit-learn 0.18 checks
from distutils.version import LooseVersion as Version
from sklearn import __version__ as sklearn_version

<font size="5">欠損データの扱い</font>

In [4]:
import pandas as pd
from io import StringIO

csv_data = '''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''

df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [5]:
#各行毎の欠損データをカウント
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

In [6]:
#欠損データを含む行を削除
df.dropna()

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [7]:
#欠損データを含む列を削除
df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


In [8]:
#全て欠損データを含む行を削除
df.dropna(how='all') 

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [9]:
#特定の列に欠損データを含む行を削除
df.dropna(subset=['C'])

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


<font size="5">欠損データの補完</font>

In [10]:
from sklearn.preprocessing import Imputer

imr = Imputer(missing_values='NaN', strategy='mean', axis=0)
imr = imr.fit(df)
imputed_data = imr.transform(df.values)
imputed_data

array([[  1. ,   2. ,   3. ,   4. ],
       [  5. ,   6. ,   7.5,   8. ],
       [ 10. ,  11. ,  12. ,   6. ]])

<font size="5">カテゴリデータの扱い</font>

In [11]:
import pandas as pd

df = pd.DataFrame([['green', 'M', 10.1, 'class1'],
                   ['red', 'L', 13.5, 'class2'],
                   ['blue', 'XL', 15.3, 'class1']])

df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


In [12]:
#順序特徴量を設定する
size_mapping = {'XL': 3,
                'L': 2,
                'M': 1}

df['size'] = df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


In [13]:
inv_size_mapping = {v: k for k, v in size_mapping.items()} #内包的表現
df['size'].map(inv_size_mapping) #sizeをキー値として呼び出し、ラベルを表示

0     M
1     L
2    XL
Name: size, dtype: object

<font size="5">クラスラベルの符号化</font>

In [14]:
import numpy as np

class_mapping = {label: idx for idx, label in enumerate(np.unique(df['classlabel']))}
class_mapping

{'class1': 0, 'class2': 1}

In [15]:
df['classlabel'] = df['classlabel'].map(class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,0
1,red,2,13.5,1
2,blue,3,15.3,0


In [16]:
# 符号化したクラスからラベルを再度割り当てる
inv_class_mapping = {v: k for k, v in class_mapping.items()}
df['classlabel'] = df['classlabel'].map(inv_class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


In [17]:
from sklearn.preprocessing import LabelEncoder

#ラベルエンコーダのインスタンシエーション
class_le = LabelEncoder()

#クラスラベルに整数を割り当て
y = class_le.fit_transform(df['classlabel'].values)
y

array([0, 1, 0], dtype=int64)

In [18]:
#クラスラベルに戻す
class_le.inverse_transform(y)

array(['class1', 'class2', 'class1'], dtype=object)

In [19]:
# color, size, price を抽出
X = df[['color', 'size', 'price']].values

# blue
# green
# red -> 2
color_le = LabelEncoder()
X[:, 0] = color_le.fit_transform(X[:, 0])
X

array([[1, 1, 10.1],
       [2, 2, 13.5],
       [0, 3, 15.3]], dtype=object)